# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

# Geração de scripts de TV
Neste projeto, você gerará seus próprios scripts de TV [Simpsons] (https://en.wikipedia.org/wiki/The_Simpsons) usando RNNs. Você estará usando parte do [conjunto de dados Simpsons] (https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) de scripts de 27 temporadas. A Rede Neural que você irá construir irá gerar um novo roteiro de TV para uma cena na [Moe's Tavern] (https://simpsonswiki.com/wiki/Moe's_Tavern).
## Obtém os dados
Os dados já são fornecidos para você. Você estará usando um subconjunto do conjunto de dados original. Consiste apenas nas cenas da Taverna de Moe. Isso não inclui outras versões da taverna, como "Moe's Cavern", "Flame Moe's", "Uncle Moe's Family Feed-Bag", etc.


In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

## Explore os dados
Brinque com `view_sentence_range` para ver diferentes partes dos dados.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

## Implementar funções de pré-processamento
A primeira coisa a fazer em qualquer conjunto de dados é o pré-processamento. Implemente as seguintes funções de pré-processamento abaixo:
- Tabela de pesquisa
- Tokenize Punctuation

### Tabela de pesquisa
Para criar uma incorporação de palavras, primeiro você precisa transformar as palavras em ids. Nesta função, crie dois dicionários:
- Dicionário para ir das palavras para um id, vamos chamar `vocab_to_int`
- Dicionário para ir do id para a palavra, vamos chamar `int_to_vocab`

Devolve estes dicionários na seguinte tupla `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    from collections import Counter
    counts=Counter(text)
    vocab=sorted(counts,key=counts.get, reverse=True)
    # TODO: Implement Function
    vocab_to_int={word:index for index,word in enumerate(vocab,1)}
    int_to_vocab={index:word for index,word in enumerate(vocab,1)}
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

### Tokenize Punctuation
Nós dividiremos o script em uma matriz de palavras usando espaços como delimitadores. No entanto, pontuações como períodos e pontos de exclamação tornam difícil para a rede neural distinguir entre a palavra "bye" e "bye!".

Implemente a função `token_lookup` para retornar um comando que será usado para simbolizar símbolos como"! " em "|| Exclamation_Mark ||". Crie um dicionário para os seguintes símbolos, onde o símbolo é a chave e o valor é o token:
- Período (.)
- vírgula (,)
- Aspa ( " )
- ponto e vírgula (;)
- Ponto de exclamação ( ! )
- Ponto de interrogação ( ? )
- Parênteses esquerdos (()
- Parênteses direitos ())
- traço (-)
- retorno (\ n)

Este dicionário será usado para simbolizar os símbolos e adicionar o delimitador (espaço) em torno dele. Isso separa os símbolos como sua própria palavra, facilitando a previsão da rede neural na próxima palavra. Certifique-se de não usar um token que possa ser confundido como uma palavra. Em vez de usar o token "dash", tente usar algo como "|| dash ||".



In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    return {'.':"||Period||",',':"||Comma||",'"':"||Quotation_Mark||",';':"||Semicolon||",'!':"||Exclamation_mark||",'?':"||Question_mark||",'(':"||Left_Parentheses||",')':"||Right_Parentheses||",'\n':"||Dash||",'--':"||Return||"}

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

## Pré-processe todos os dados e salve-os
A execução da célula de código abaixo irá pré-processar todos os dados e salvá-los no arquivo.


In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

# Check Point
Este é seu primeiro checkpoint. Se você decidir voltar a este notebook ou precisar reiniciar o notebook, poderá começar a partir daqui. Os dados pré-processados foram salvos no disco.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

## Construa a Rede Neural
Você construirá os componentes necessários para construir um RNN implementando as seguintes funções abaixo:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Verifique a versão do TensorFlow e acesse o GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.8.0


C:\Users\ITALO\Anaconda3\envs\ITALO\lib\site-packages\ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

### Entrada
Implemente a função `get_inputs ()` para criar Placeholders do TF para a Rede Neural. Deve criar os seguintes marcadores de posição:
- Espaço reservado para entrada de texto chamado "input" usando o parâmetro [TF Placeholder] (https://www.tensorflow.org/api_docs/python/tf/placeholder) `name`.
- Espaço reservado para alvos
- Espaço reservado para Learning Rate

Retorna os placeholders na seguinte tupla `(Input, Targets, LearningRate)`

In [3]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    Input=tf.placeholder(tf.int32,shape=(None,None),name='input')
    Targets=tf.placeholder(tf.int32,shape=(None,None),name='targets')
    LearningRate=tf.placeholder(tf.float32,name='learning_rate')
    return (Input, Targets, LearningRate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`


### Construir Célula RNN e Inicializar
Empilhe uma ou mais [`BasicLSTMCells`] (https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) em um [` MultiRNNCell`] (https://www.tensorflow.org/ api_docs / python / tf / contrib / rnn / MultiRNNCell).
- O tamanho do Rnn deve ser definido usando `rnn_size`
- Initalize Cell State usando a função [`zero_state ()`] (https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) do MultiRNNCell
     - Aplique o nome "initial_state" ao estado inicial usando [`tf.identity ()`] (https://www.tensorflow.org/api_docs/python/tf/identity)

Retorna a célula e o estado inicial na seguinte tupla `(Cell, InitialState)`


In [4]:
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

    return drop


def build_lstm(lstm_size, batch_size, num_layers, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state=tf.identity(initial_state,name="initial_state")
    
    return cell, initial_state


In [5]:
def get_init_cell(batch_size, rnn_size, num_layers=2, keep_prob=0.7):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    cell, initial_state=build_lstm(rnn_size, batch_size,num_layers, keep_prob)
    return (cell, initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.


### Incorporação de palavras
Aplique a incorporação ao `input_data` usando o TensorFlow. Devolva a sequência incorporada.

In [6]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size,embed_dim),-1,1))
    embed = tf.nn.embedding_lookup(embedding,input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

### Construir RNN
Você criou uma célula RNN na função `get_init_cell ()`. Hora de usar a célula para criar um RNN.
- Construa o RNN usando o [`tf.nn.dynamic_rnn ()`] (https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
  - Aplique o nome "final_state" ao estado final usando [`tf.identity ()`] (https://www.tensorflow.org/api_docs/python/tf/identity)

Retorna as saídas e o estado final_state na seguinte tupla `(Saídas, FinalState)`

In [7]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs,dtype=tf.float32)
    final_state=tf.identity(final_state,name="final_state")
    return (outputs, final_state)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

### Construa a Rede Neural
Aplique as funções implementadas acima para:
- Aplicar incorporação ao `input_data` usando sua função` get_embed (input_data, vocab_size, embed_dim) `.
- Construa RNN usando `cell` e sua função` build_rnn (cell, inputs) `.
- Aplique uma camada totalmente conectada com uma ativação linear e `vocab_size` como o número de saídas.

Retorna os logits e o estado final na seguinte tupla (Logits, FinalState)


In [8]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embeded=get_embed(input_data, vocab_size, embed_dim)
    output_build_rnn=build_rnn(cell, embeded)
    outputs=output_build_rnn[0]
    final_state=output_build_rnn[1]
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
   
    return (logits, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

### Lotes
Implemente `get_batches` para criar lotes de entrada e destinos usando` int_text`. Os lotes devem ser um array Numpy com a forma `(número de lotes, 2, tamanho do lote, comprimento da seqüência)`. Cada lote contém dois elementos:
- O primeiro elemento é um único lote de ** entrada ** com a forma `[tamanho do lote, comprimento da sequência]`
- O segundo elemento é um único lote de ** destinos ** com a forma `[tamanho do lote, comprimento da sequência]`

Se você não puder preencher o último lote com dados suficientes, elimine o último lote.

Por exemplo, `get_batches ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2) `retornaria uma matriz Numpy dos seguintes:
`` `
[
  # Primeiro lote
  [
    # Lote de entrada
    [[1 2], [7 8], [13 14]]
    # Lote de alvos
    [[2 3], [8 9], [14 15]]
  ]

  # Segundo lote
  [
    # Lote de entrada
    [[3 4], [9 10], [15 16]]
    # Lote de alvos
    [[4 5], [10 11], [16 17]]
  ]

  # Terceiro lote
  [
    # Lote de entrada
    [[5 6], [11 12], [17 18]]
    # Lote de alvos
    [[6 7], [12 13], [18 1]]
  ]
]
`` `

Observe que o último valor de destino no último lote é o primeiro valor de entrada do primeiro lote. Neste caso, `1`. Essa é uma técnica comum usada na criação de lotes de sequências, embora não seja intuitiva.

In [9]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    characters_per_batch = batch_size * seq_length
    #n_batches = len(int_text)//characters_per_batch
    n_batches = len(int_text)//characters_per_batch
    int_text = int_text[:n_batches*characters_per_batch]
    xx=[]
    xx=int_text[1:]
    xx.append(int_text[0])
    int_text=np.array(int_text)
    xx=np.array(xx)
    int_text = int_text.reshape((batch_size,-1))
    xx = xx.reshape((batch_size,-1))
    output=[]
    for n in range(0, int_text.shape[1], seq_length):
        # The features
        x = int_text[:,n:n+seq_length]
        # The targets, shifted by one
        y = xx[:,n:n+seq_length]
        x,y=x.tolist(),y.tolist()
        d=[]
        d.append(x)
        d.append(y)
        output.append(d)
    return np.array(np.array(output))
        

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

## Treinamento em Redes Neurais
### Hyperparameters
Ajuste os seguintes parâmetros:

- Defina `num_epochs` como o número de épocas.
- Configure `batch_size` para o tamanho do lote.
- Defina `rnn_size` para o tamanho dos RNNs.
- Configure `embed_dim` para o tamanho da incorporação.
- Defina `seq_length` como o comprimento da sequência.
- Configure `learning_rate` para a taxa de aprendizado.
- Defina `show_every_n_batches` com o número de lotes que a rede neural deve imprimir.


In [10]:
# Number of Epochs
num_epochs = 300
# Batch Size
batch_size = 32
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 300
# Sequence Length
seq_length = 50
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 5

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

### Construir o gráfico
Construa o gráfico usando a rede neural que você implementou.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)

    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

## Trem
Treine a rede neural nos dados pré-processados. Se você tiver dificuldade em obter uma boa perda, verifique os [fóruns] (https://discussions.udacity.com/) para ver se alguém está tendo o mesmo problema.

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/43   train_loss = 8.822
Epoch   0 Batch    5/43   train_loss = 7.084
Epoch   0 Batch   10/43   train_loss = 6.456
Epoch   0 Batch   15/43   train_loss = 6.434
Epoch   0 Batch   20/43   train_loss = 6.335
Epoch   0 Batch   25/43   train_loss = 6.512
Epoch   0 Batch   30/43   train_loss = 6.314
Epoch   0 Batch   35/43   train_loss = 6.316
Epoch   0 Batch   40/43   train_loss = 6.146
Epoch   1 Batch    2/43   train_loss = 6.235
Epoch   1 Batch    7/43   train_loss = 6.119
Epoch   1 Batch   12/43   train_loss = 6.121
Epoch   1 Batch   17/43   train_loss = 6.147
Epoch   1 Batch   22/43   train_loss = 6.061
Epoch   1 Batch   27/43   train_loss = 6.233
Epoch   1 Batch   32/43   train_loss = 6.341
Epoch   1 Batch   37/43   train_loss = 6.214
Epoch   1 Batch   42/43   train_loss = 6.069
Epoch   2 Batch    4/43   train_loss = 6.176
Epoch   2 Batch    9/43   train_loss = 6.073
Epoch   2 Batch   14/43   train_loss = 6.178
Epoch   2 Batch   19/43   train_loss = 6.194
Epoch   2 

Epoch  21 Batch   12/43   train_loss = 4.494
Epoch  21 Batch   17/43   train_loss = 4.588
Epoch  21 Batch   22/43   train_loss = 4.472
Epoch  21 Batch   27/43   train_loss = 4.601
Epoch  21 Batch   32/43   train_loss = 4.684
Epoch  21 Batch   37/43   train_loss = 4.544
Epoch  21 Batch   42/43   train_loss = 4.378
Epoch  22 Batch    4/43   train_loss = 4.520
Epoch  22 Batch    9/43   train_loss = 4.486
Epoch  22 Batch   14/43   train_loss = 4.584
Epoch  22 Batch   19/43   train_loss = 4.456
Epoch  22 Batch   24/43   train_loss = 4.361
Epoch  22 Batch   29/43   train_loss = 4.442
Epoch  22 Batch   34/43   train_loss = 4.523
Epoch  22 Batch   39/43   train_loss = 4.455
Epoch  23 Batch    1/43   train_loss = 4.434
Epoch  23 Batch    6/43   train_loss = 4.256
Epoch  23 Batch   11/43   train_loss = 4.505
Epoch  23 Batch   16/43   train_loss = 4.437
Epoch  23 Batch   21/43   train_loss = 4.307
Epoch  23 Batch   26/43   train_loss = 4.539
Epoch  23 Batch   31/43   train_loss = 4.396
Epoch  23 

Epoch  42 Batch   24/43   train_loss = 3.407
Epoch  42 Batch   29/43   train_loss = 3.466
Epoch  42 Batch   34/43   train_loss = 3.463
Epoch  42 Batch   39/43   train_loss = 3.422
Epoch  43 Batch    1/43   train_loss = 3.343
Epoch  43 Batch    6/43   train_loss = 3.264
Epoch  43 Batch   11/43   train_loss = 3.449
Epoch  43 Batch   16/43   train_loss = 3.472
Epoch  43 Batch   21/43   train_loss = 3.370
Epoch  43 Batch   26/43   train_loss = 3.516
Epoch  43 Batch   31/43   train_loss = 3.391
Epoch  43 Batch   36/43   train_loss = 3.402
Epoch  43 Batch   41/43   train_loss = 3.274
Epoch  44 Batch    3/43   train_loss = 3.350
Epoch  44 Batch    8/43   train_loss = 3.443
Epoch  44 Batch   13/43   train_loss = 3.415
Epoch  44 Batch   18/43   train_loss = 3.366
Epoch  44 Batch   23/43   train_loss = 3.359
Epoch  44 Batch   28/43   train_loss = 3.354
Epoch  44 Batch   33/43   train_loss = 3.392
Epoch  44 Batch   38/43   train_loss = 3.341
Epoch  45 Batch    0/43   train_loss = 3.327
Epoch  45 

Epoch  63 Batch   36/43   train_loss = 2.455
Epoch  63 Batch   41/43   train_loss = 2.508
Epoch  64 Batch    3/43   train_loss = 2.473
Epoch  64 Batch    8/43   train_loss = 2.488
Epoch  64 Batch   13/43   train_loss = 2.516
Epoch  64 Batch   18/43   train_loss = 2.488
Epoch  64 Batch   23/43   train_loss = 2.468
Epoch  64 Batch   28/43   train_loss = 2.458
Epoch  64 Batch   33/43   train_loss = 2.473
Epoch  64 Batch   38/43   train_loss = 2.487
Epoch  65 Batch    0/43   train_loss = 2.454
Epoch  65 Batch    5/43   train_loss = 2.439
Epoch  65 Batch   10/43   train_loss = 2.476
Epoch  65 Batch   15/43   train_loss = 2.545
Epoch  65 Batch   20/43   train_loss = 2.493
Epoch  65 Batch   25/43   train_loss = 2.406
Epoch  65 Batch   30/43   train_loss = 2.508
Epoch  65 Batch   35/43   train_loss = 2.520
Epoch  65 Batch   40/43   train_loss = 2.401
Epoch  66 Batch    2/43   train_loss = 2.400
Epoch  66 Batch    7/43   train_loss = 2.465
Epoch  66 Batch   12/43   train_loss = 2.332
Epoch  66 

Epoch  85 Batch    5/43   train_loss = 1.888
Epoch  85 Batch   10/43   train_loss = 1.736
Epoch  85 Batch   15/43   train_loss = 1.928
Epoch  85 Batch   20/43   train_loss = 1.787
Epoch  85 Batch   25/43   train_loss = 1.780
Epoch  85 Batch   30/43   train_loss = 1.884
Epoch  85 Batch   35/43   train_loss = 1.874
Epoch  85 Batch   40/43   train_loss = 1.772
Epoch  86 Batch    2/43   train_loss = 1.682
Epoch  86 Batch    7/43   train_loss = 1.810
Epoch  86 Batch   12/43   train_loss = 1.628
Epoch  86 Batch   17/43   train_loss = 1.809
Epoch  86 Batch   22/43   train_loss = 1.730
Epoch  86 Batch   27/43   train_loss = 1.770
Epoch  86 Batch   32/43   train_loss = 1.788
Epoch  86 Batch   37/43   train_loss = 1.766
Epoch  86 Batch   42/43   train_loss = 1.734
Epoch  87 Batch    4/43   train_loss = 1.773
Epoch  87 Batch    9/43   train_loss = 1.756
Epoch  87 Batch   14/43   train_loss = 1.767
Epoch  87 Batch   19/43   train_loss = 1.737
Epoch  87 Batch   24/43   train_loss = 1.814
Epoch  87 

...

Epoch 212 Batch   34/43   train_loss = 0.267
Epoch 212 Batch   39/43   train_loss = 0.300
Epoch 213 Batch    1/43   train_loss = 0.276
Epoch 213 Batch    6/43   train_loss = 0.278
Epoch 213 Batch   11/43   train_loss = 0.261
Epoch 213 Batch   16/43   train_loss = 0.285
Epoch 213 Batch   21/43   train_loss = 0.324
Epoch 213 Batch   26/43   train_loss = 0.289
Epoch 213 Batch   31/43   train_loss = 0.304
Epoch 213 Batch   36/43   train_loss = 0.293
Epoch 213 Batch   41/43   train_loss = 0.290
Epoch 214 Batch    3/43   train_loss = 0.298
Epoch 214 Batch    8/43   train_loss = 0.266
Epoch 214 Batch   13/43   train_loss = 0.298
Epoch 214 Batch   18/43   train_loss = 0.284
Epoch 214 Batch   23/43   train_loss = 0.300
Epoch 214 Batch   28/43   train_loss = 0.267
Epoch 214 Batch   33/43   train_loss = 0.299
Epoch 214 Batch   38/43   train_loss = 0.317
Epoch 215 Batch    0/43   train_loss = 0.277
Epoch 215 Batch    5/43   train_loss = 0.293
Epoch 215 Batch   10/43   train_loss = 0.255
Epoch 215 

Epoch 234 Batch    3/43   train_loss = 0.253
Epoch 234 Batch    8/43   train_loss = 0.209
Epoch 234 Batch   13/43   train_loss = 0.259
Epoch 234 Batch   18/43   train_loss = 0.231
Epoch 234 Batch   23/43   train_loss = 0.256
Epoch 234 Batch   28/43   train_loss = 0.227
Epoch 234 Batch   33/43   train_loss = 0.251
Epoch 234 Batch   38/43   train_loss = 0.257
Epoch 235 Batch    0/43   train_loss = 0.254
Epoch 235 Batch    5/43   train_loss = 0.241
Epoch 235 Batch   10/43   train_loss = 0.247
Epoch 235 Batch   15/43   train_loss = 0.235
Epoch 235 Batch   20/43   train_loss = 0.200
Epoch 235 Batch   25/43   train_loss = 0.217
Epoch 235 Batch   30/43   train_loss = 0.253
Epoch 235 Batch   35/43   train_loss = 0.284
Epoch 235 Batch   40/43   train_loss = 0.250
Epoch 236 Batch    2/43   train_loss = 0.241
Epoch 236 Batch    7/43   train_loss = 0.257
Epoch 236 Batch   12/43   train_loss = 0.215
Epoch 236 Batch   17/43   train_loss = 0.235
Epoch 236 Batch   22/43   train_loss = 0.233
Epoch 236 

Epoch 255 Batch   15/43   train_loss = 0.233
Epoch 255 Batch   20/43   train_loss = 0.177
Epoch 255 Batch   25/43   train_loss = 0.170
Epoch 255 Batch   30/43   train_loss = 0.196
Epoch 255 Batch   35/43   train_loss = 0.225
Epoch 255 Batch   40/43   train_loss = 0.227
Epoch 256 Batch    2/43   train_loss = 0.198
Epoch 256 Batch    7/43   train_loss = 0.242
Epoch 256 Batch   12/43   train_loss = 0.202
Epoch 256 Batch   17/43   train_loss = 0.228
Epoch 256 Batch   22/43   train_loss = 0.189
Epoch 256 Batch   27/43   train_loss = 0.226
Epoch 256 Batch   32/43   train_loss = 0.225
Epoch 256 Batch   37/43   train_loss = 0.203
Epoch 256 Batch   42/43   train_loss = 0.225
Epoch 257 Batch    4/43   train_loss = 0.196
Epoch 257 Batch    9/43   train_loss = 0.210
Epoch 257 Batch   14/43   train_loss = 0.211
Epoch 257 Batch   19/43   train_loss = 0.199
Epoch 257 Batch   24/43   train_loss = 0.212
Epoch 257 Batch   29/43   train_loss = 0.219
Epoch 257 Batch   34/43   train_loss = 0.188
Epoch 257 

Epoch 276 Batch   27/43   train_loss = 0.190
Epoch 276 Batch   32/43   train_loss = 0.158
Epoch 276 Batch   37/43   train_loss = 0.186
Epoch 276 Batch   42/43   train_loss = 0.169
Epoch 277 Batch    4/43   train_loss = 0.171
Epoch 277 Batch    9/43   train_loss = 0.179
Epoch 277 Batch   14/43   train_loss = 0.171
Epoch 277 Batch   19/43   train_loss = 0.155
Epoch 277 Batch   24/43   train_loss = 0.200
Epoch 277 Batch   29/43   train_loss = 0.171
Epoch 277 Batch   34/43   train_loss = 0.161
Epoch 277 Batch   39/43   train_loss = 0.181
Epoch 278 Batch    1/43   train_loss = 0.172
Epoch 278 Batch    6/43   train_loss = 0.150
Epoch 278 Batch   11/43   train_loss = 0.166
Epoch 278 Batch   16/43   train_loss = 0.159
Epoch 278 Batch   21/43   train_loss = 0.169
Epoch 278 Batch   26/43   train_loss = 0.199
Epoch 278 Batch   31/43   train_loss = 0.168
Epoch 278 Batch   36/43   train_loss = 0.158
Epoch 278 Batch   41/43   train_loss = 0.196
Epoch 279 Batch    3/43   train_loss = 0.172
Epoch 279 

Epoch 297 Batch   39/43   train_loss = 0.160
Epoch 298 Batch    1/43   train_loss = 0.143
Epoch 298 Batch    6/43   train_loss = 0.161
Epoch 298 Batch   11/43   train_loss = 0.152
Epoch 298 Batch   16/43   train_loss = 0.149
Epoch 298 Batch   21/43   train_loss = 0.162
Epoch 298 Batch   26/43   train_loss = 0.161
Epoch 298 Batch   31/43   train_loss = 0.133
Epoch 298 Batch   36/43   train_loss = 0.152
Epoch 298 Batch   41/43   train_loss = 0.161
Epoch 299 Batch    3/43   train_loss = 0.150
Epoch 299 Batch    8/43   train_loss = 0.132
Epoch 299 Batch   13/43   train_loss = 0.150
Epoch 299 Batch   18/43   train_loss = 0.164
Epoch 299 Batch   23/43   train_loss = 0.141
Epoch 299 Batch   28/43   train_loss = 0.146
Epoch 299 Batch   33/43   train_loss = 0.160
Epoch 299 Batch   38/43   train_loss = 0.165
Model Trained and Saved


## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

## Save Parameters
Salve `seq_length` e` save_dir` para gerar um novo script de TV.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

## Implementar Gerar Funções
### Obtenha Tensores
Obtenha tensores de `loaded_graph` usando a função [` get_tensor_by_name () `] (https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name). Obtenha os tensores usando os seguintes nomes:
- "entrada: 0"
- "initial_state: 0"
- "final_state: 0"
- "probs: 0"

Retorna os tensores na seguinte tupla `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)`

In [17]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    return (loaded_graph.get_tensor_by_name("input:0"), loaded_graph.get_tensor_by_name("initial_state:0"), loaded_graph.get_tensor_by_name("final_state:0"), loaded_graph.get_tensor_by_name("probs:0"))


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

### Escolher palavra
Implemente a função `pick_word ()` para selecionar a próxima palavra usando `probabilities`.

In [18]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    index=np.random.choice(a=list(int_to_vocab.keys()),p=probabilities)
    return int_to_vocab[index]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

## Gerar script de TV
Isso irá gerar o script de TV para você. Defina `gen_length` com o comprimento do script de TV que você deseja gerar.

In [21]:
gen_length = 400
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:) cocktail wanna are in carl_carlson: yeah(our yourself the longer the three second the arms the la spending, never didn't but a who check", for not
, no(outside furious go spending looks how, shrugs oh(light guys the stories the might lousy the chuckles the little_man: fellas(surgery and dinner right more" strong barn
, barney_gumble: barney
about!)) you(once the man: moe's(understand how from it keep bury hey me
give that this this this this this this this this this moe_szyslak: just only family
, you know
, barney_gumble: for beer a remember oh look moe_szyslak: you are man
, for can you into homer_simpson:
, barney_gumble: but you then have be c
what you time have and struggling yeah a wait by our c
what you have media
things you are be in drunk
, lenny_leonard: ever(once the because from changed your throw(go the forget your much derisive listen
, of it
, well(happened here's hey
would! even it we're a felt
?)) you(true 

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.

# O roteiro de TV é absurdo
Tudo bem se o roteiro da TV não fizer sentido. Nós treinamos com menos de um megabyte de texto. Para obter bons resultados, você terá que usar um vocabulário menor ou obter mais dados. Felizmente há mais dados! Como mencionamos no início deste projeto, este é um subconjunto de [outro conjunto de dados] (https://www.kaggle.com/wcukierski/the-simpsons-by-the-data). Nós não fizemos você treinar em todos os dados, porque isso levaria muito tempo. No entanto, você está livre para treinar sua rede neural em todos os dados. Depois de concluir o projeto, é claro.
# Enviando este projeto
Ao enviar este projeto, certifique-se de executar todas as células antes de salvar o bloco de anotações. Salve o arquivo do notebook como "dlnd_tv_script_generation.ipynb" e salve-o como um arquivo HTML em "File" -> "Download as". Inclua os arquivos "helper.py" e "problem_unittests.py" no seu envio.